In [1]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import librosa
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    Conv1D,
    LSTM,
    Bidirectional,
    Dropout,
    BatchNormalization,
    Dense,
    MaxPooling1D,
    concatenate,
    GlobalAveragePooling1D,
    Flatten,
)

from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import save_model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix, roc_curve

import seaborn as sns
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
data_path = "/content/DATASET-balanced.csv"
df = pd.read_csv(data_path)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/DATASET-balanced.csv'

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='LABEL')

for p in plt.gca().patches:
    plt.text(p.get_x() + p.get_width() / 2, p.get_height() + 0.1, int(p.get_height()),
             ha='center', va='bottom')

plt.show()

In [ ]:
label_encoder = LabelEncoder()
df['LABEL'] = label_encoder.fit_transform(df['LABEL'])

In [ ]:
y = df['LABEL']
X = df.drop('LABEL', axis = 1)
for column in X:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=column, hue='LABEL', kde=True, palette=['blue', 'orange'])
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.legend(labels=['FAKE', 'REAL'])
    plt.show()

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(20, 8))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot = True)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
def prepare_data(X, window_size = 10):

    data = []

    for i in range(len(X)):
        row = X.iloc[i].values
        row_data = []
        for j in range(len(row) - window_size):
            window = row[j : j + window_size]
            row_data.append(window)
        data.append(row_data)

    return np.array(data)

In [ ]:
X.iloc[:, :] = MinMaxScaler().fit_transform(X)
new_X = prepare_data(X, window_size = 5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y.values, test_size=0.2, shuffle = True, stratify = y.values, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, stratify = y_train, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, Dense
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

model = Sequential()

input_shape = (X_train.shape[1], X_train.shape[2])
model.add(Input(shape=input_shape))

# First LSTM layer
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))

# Third LSTM layer
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))

# Fourth LSTM layer
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))

# First Dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Second Dense layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

In [ ]:
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot= True, fmt='.4g')
ax.xaxis.set_ticklabels(['Fake', 'Real'])
ax.yaxis.set_ticklabels(['Fake', 'Real'])
plt.show()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

callback = [
    ModelCheckpoint(filepath='model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, baseline=None, restore_best_weights=True, verbose=1)
]

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=60, validation_data = (X_val, y_val), callbacks = callback)

In [ ]:
y_pred = np.round(model.predict(X_test).flatten())


In [ ]:
model = load_model('model.keras')

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

In [ ]:
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
EPOCHS = len(acc)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
from scipy.ndimage import gaussian_filter1d

sacc = gaussian_filter1d(acc, sigma=2)
sval_acc = gaussian_filter1d(val_acc, sigma=2)

sloss = gaussian_filter1d(loss, sigma=2)
sval_loss = gaussian_filter1d(val_loss, sigma=2)
plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), sacc, label='Training Accuracy')
plt.plot(range(EPOCHS), sval_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy (Smoothed)')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), sloss, label='Training Loss')
plt.plot(range(EPOCHS), sval_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss (Smoothed)')
plt.show()

In [ ]:
import librosa
import numpy as np
import pandas as pd

# ==== INPUTS ====
audio_path = "/content/FAKE.mp3"  # Change this to your file path
label_value = "REAL"  # Set this to your label (e.g., "REAL" or "FAKE")

# ==== LOAD AUDIO ====
y, sr = librosa.load(audio_path, sr=None)

# ==== EXTRACT FEATURES ====
chroma = librosa.feature.chroma_stft(y=y, sr=sr)
chroma_stft_mean = np.mean(chroma, axis=0).reshape(-1, 1)

rms = librosa.feature.rms(y=y).T
spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).T
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr).T
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).T
zcr = librosa.feature.zero_crossing_rate(y).T
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T

# ==== TRIM ALL TO SAME FRAME COUNT ====
min_frames = min(
    chroma_stft_mean.shape[0],
    rms.shape[0],
    spec_centroid.shape[0],
    spec_bw.shape[0],
    rolloff.shape[0],
    zcr.shape[0],
    mfccs.shape[0]
)

chroma_stft_mean = chroma_stft_mean[:min_frames]
rms = rms[:min_frames]
spec_centroid = spec_centroid[:min_frames]
spec_bw = spec_bw[:min_frames]
rolloff = rolloff[:min_frames]
zcr = zcr[:min_frames]
mfccs = mfccs[:min_frames]

# ==== COMBINE FEATURES ====
features = np.hstack((
    chroma_stft_mean,   # 1
    rms,                # 1
    spec_centroid,      # 1
    spec_bw,            # 1
    rolloff,            # 1
    zcr,                # 1
    mfccs               # 20
))  # Final shape: (min_frames, 26)

# ==== CREATE COLUMN NAMES ====
column_names = [
    "chroma_stft",
    "rms",
    "spectral_centroid",
    "spectral_bandwidth",
    "rolloff",
    "zero_crossing_rate"
] + [f"mfcc_{i+1}" for i in range(20)]

# ==== CREATE DATAFRAME ====
df = pd.DataFrame(features, columns=column_names)

# ==== ADD LABEL COLUMN ====
df["LABEL"] = label_value

# ==== SAVE TO CSV ====
df.to_csv("FAKE", index=False)

print("✅ Saved 'final_audio_features_27cols.csv' with 26 features + 1 label column.")

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("/content/model.keras")  # Update path if needed

# Load extracted features
df = pd.read_csv("/content/FAKE")

# Select only the first 5 feature columns (excluding the label)
X = df.drop("LABEL", axis=1).iloc[:, :5].values  # shape: (timesteps, 5)

# Reshape to 3D: (1, timesteps, 5)
X = X.reshape(1, X.shape[0], X.shape[1])  # (1, frames, 5)

# Predict
prediction = model.predict(X)

print("Prediction output:", prediction)

# If binary classification (e.g., real vs fake):
if prediction[0][0] > 0.5:
    print("🎭 FAKE Audio")
else:
    print("🎤 REAL Audio")


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model, Model

# Load your trained model
model = load_model("/content/model.keras")

# Build an embedding extractor up to the second-to-last layer

# Load features from your single test audio
df_test = pd.read_csv("/content/FAKE")
X = df_test.drop("LABEL", axis=1).iloc[:, :5].values
X = X.reshape(1, X.shape[0], X.shape[1])

# Extract embeddings and reduce to a single vector by averaging over frames
emb_test = model.predict(X).mean(axis=1).flatten()

# Load a single real/reference audio sample for baseline comparison
df_real = pd.read_csv("/content/audio_features")
X_ref = df_real.drop("LABEL", axis=1).iloc[:, :5].values
X_ref = X_ref.reshape(1, X_ref.shape[0], X_ref.shape[1])
emb_real = model.predict(X_ref).mean(axis=1).flatten()

# Compute Euclidean distance between test and reference embeddings
dist = np.linalg.norm(emb_test - emb_real)
print(f"Euclidean distance to real baseline: {dist:.4f}")

# Print result using threshold comparison
threshold = 0.5  # adjust based on your validation set
if dist > threshold:
    print("🎭 FAKE Audio")
else:
    print("🎤 REAL Audio")
